## Initial Set-Up

We'll be using the [resize (cv2) function](https://www.opencv.org.cn/opencvdoc/2.3.2/html/modules/imgproc/doc/geometric_transformations.html#cv2.resize)

In [19]:
!pip install opencv-python
!pip install tqdm
!pip install tensorflow
!pip install pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-t7vnc5k8/pytorch_b1bc3931ede8483e9f446b5460db9bdb/setup.py'"'"'; __file__='"'"'/tmp/pip-install-t7vnc5k8/pytorch_b1bc3931ede8483e9f446b5460db9bdb/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup(

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm # progress bar

In [ ]:
from genericpath import isdir
directory = "../mirflickr"
count = 0
# iterate through each file in the dir and push file data into imgs list
if not os.path.isdir(directory):
   os.makedirs(directory)

for filename in tqdm(os.listdir(directory)):
    filepath = os.path.join(directory, filename)
    img = cv2.imread(filepath)
    
    if '.jpg' not in filename:
        continue
    
    try:
        hr_output = './imgs-hr' + filename
        lr_output = './imgs-lr/' + filename
        hr_img = cv2.resize(img, np.array([128, 128]))
        lr_img = cv2.resize(img, np.array([32, 32]))

        cv2.imwrite(hr_output, hr_img)
        cv2.imwrite(lr_output, lr_img)
    except Exception as e:
        print('failed to resize ', hr_output, e)
        print(filename)
        print(img)

0it [00:00, ?it/s]


In [20]:
# imports
import tensorflow as tf
from keras import Input
from keras.applications import VGG19
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, PReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from tensorflow.keras.models import load_model

ModuleNotFoundError: ignored

In [17]:
# create generator network
def create_generator_network():
    inputs = Input(shape=(32, 32, 3))

    # original 2 layers
    a = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(inputs)
    a = PReLU()(a)

    # keep track of a to sum it into the result of the residiual block methods
    orig = a

    for i in range(0, 5):
        a = get_residual_block(x, a)

    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(a)
    x = BatchNormalization()(x)

    a = Add()([x, orig])

    for i in range(0, 2):
        # pixel shuffle
        a = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(a)
        a = UpSampling2D()(a)
        a = PReLU()(a)

    a = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(a)

    return a

def get_residual_block(x, a):
    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(a)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(x)
    x = BatchNormalization()(x)
    a = Add()([x, a])

    return a

In [18]:
# create discriminator network
def create_discriminator_network():
    inputs = Input(shape=(32, 32, 3))

    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(inputs)
    x = LeakyReLU()(x)

    for i in range(0, 7):
      x = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3], activation='relu')(x)
      x = BatchNormalization()()(x)
      x = LeakyReLU()(x)

    x = Dense(1024, activation="relu")(x)
    x = LeakyReLU()(x)
    x = Dense(1, activation="relu")(x)
    x = tf.keras.activations.sigmoid(x)

    return x